# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). Используйте метрику BLEU. Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [END] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах!

### Импорты

In [1]:
#!apt-get install unzip
#!pip install torch==2.0.1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
#! pip install keras==3.1.1

In [3]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import torch
print(keras.__version__)
print(torch.__version__)

3.1.1
2.0.1+cu117


In [4]:
#%pip install tokenizers matplotlib scikit-learn

In [5]:
import tensorflow as tf
import keras
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

### Данные

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [6]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2024-04-04 11:32:21--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru.1’

opus.en-ru-train.ru 100%[===================>] 115.72M  92.4MB/s    in 1.3s    

2024-04-04 11:32:22 (92.4 MB/s) - ‘opus.en-ru-train.ru.1’ saved [121340806/121340806]

--2024-04-04 11:32:22--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en.1’

opus.en-ru-train.en 100%[===================>]  64.62M  85.8MB/s    in 0.8s    

2024-04-04 11:32:23 (85.8 MB/s) - ‘opus

In [7]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [8]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

In [9]:
# Пример перевода с английского на русский
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

### Токенизаторы

In [10]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [11]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [12]:
# c готовым декодером
# вызываем токенайзер, чтобы превратить всё в индексы
# потом делаем из индексов обратно текст
tokenizer_ru.decoder.decode(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с редким словом'

In [13]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
# tokenizer_en.save('tokenizer_en')
# tokenizer_ru.save('tokenizer_ru')

In [14]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

### Переводим текст в индексы

In [15]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids


In [16]:
# Кодируем и паддим
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

In [17]:
# выбор максимальной длины
max_len_en = np.max([len(x) for x in X_en])
max_len_ru = np.max([len(x) for x in X_ru])

In [18]:
max_len_en, max_len_ru

(17863, 19389)

In [19]:
print('Исходный язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_en], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_en], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_en], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_en], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_en], 99))

Исходный язык
50 % текстов <=  10.0
75 % текстов <=  18.0
90 % текстов <=  33.0
95 % текстов <=  45.0
99 % текстов <=  75.0


In [20]:
print('Целевой язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_ru], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_ru], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_ru], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_ru], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_ru], 99))

Целевой язык
50 % текстов <=  11.0
75 % текстов <=  20.0
90 % текстов <=  36.0
95 % текстов <=  48.0
99 % текстов <=  79.0


In [21]:
# в seq2seq длины могут быть разными
max_len_en, max_len_ru = 45, 48

In [22]:
# Вытащим тэг паддинга из словаря и будет использовать его
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

1

### Три матрицы

In [23]:
# На вход модели мы будем подавать три матрицы.

X_en = keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post',
              value=PAD_IDX)

X_ru_dec = keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1,
              padding='post', value=PAD_IDX)

In [24]:
en_sents[0]

"yeah, that's not exactly..."

In [25]:
# Первая - X_en - это индексы токенов в английских текстах (тут все как и в предыдущих семинарах)
X_en[0]

array([3280,   13, 2763,    8,   58, 2808, 5148, 2856,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1], dtype=int32)

In [26]:
# А вот русскоязычные тексты мы будем подавать через две матрицы
# - одна будет использоваться как вход для декодера (X_ru_dec),
# а вторая как правильные ответы (X_ru_out).

In [27]:
ru_sents[0]

'да, но не совсем...'

In [28]:
X_ru_dec[0]

array([   2, 2588,   15, 2589, 2513, 5362, 2643,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

In [29]:
X_ru_out[0]

array([2588,   15, 2589, 2513, 5362, 2643,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

In [30]:
# миллион примеров
X_en.shape, X_ru_out.shape

((1000000, 45), (1000000, 47))

### Разделяем на трейн и тест

In [32]:
(X_en_train, X_en_valid,
X_ru_dec_train, X_ru_dec_valid,
X_ru_out_train, X_ru_out_valid) = train_test_split(X_en,
                                                  X_ru_dec,
                                                  X_ru_out,
                                                  test_size=0.05)

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://www.tensorflow.org/text/tutorials/transformer

### Имплементация трансформера

In [33]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = keras.ops.matmul(query, keras.ops.transpose(key, axes=[0, 1, 3, 2]))

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = keras.ops.cast(key.shape[-1], torch.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / keras.ops.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = keras.ops.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = keras.ops.matmul(attention_weights, value)

    return output

In [34]:
class MultiHeadAttention(keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = keras.layers.Dense(units=d_model)
        self.key_dense = keras.layers.Dense(units=d_model)
        self.value_dense = keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = keras.ops.reshape(
            inputs, newshape=(batch_size, -1, self.num_heads, self.depth))
        return keras.ops.transpose(inputs, axes=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs.get('mask', None)
        batch_size = query.shape[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы
        # то есть просто разрезаем вектора на num_heads частей
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = keras.ops.transpose(scaled_attention, axes=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = keras.ops.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через дополнительный полносвязный слой
        # Он просто добавляет сложности нашей модели
        outputs = self.dense(concat_attention)

        return outputs

### Маски

In [35]:
# токен – pad или не pad
def create_padding_mask(x):
    mask = keras.ops.cast(keras.ops.equal(x, PAD_IDX), torch.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, None, None, :]

In [36]:
create_padding_mask([[1,2,3]]) # 1 подставилась для индекса который равен PAD_IDX

tensor([[[[1., 0., 0.]]]], device='cuda:0')

In [37]:
def create_look_ahead_mask(x):
    # эта функция немножко сложная, но суть у нее достаточно простая
    # нужно создать треугольную маску, с помощью которой мы закроем
    # для каждого токена все последующие токены
    seq_len = x.shape[1]
    ones_mask = keras.ops.ones((1, seq_len, seq_len), dtype="int32")
    row_index = keras.ops.cumsum(ones_mask, axis=-2)
    col_index = keras.ops.cumsum(ones_mask, axis=-1)
    look_ahead_mask = ~ keras.ops.greater_equal(row_index, col_index)
    padding_mask = create_padding_mask(x)
    return keras.ops.maximum(look_ahead_mask, padding_mask)

In [38]:
# для декодера нам нужно замаскировать следующие токены
# так как мы пытаемся их сгенерировать
# для этого создается вот такая маска
ones_mask = keras.ops.ones((1, 3, 3), dtype="int32")
row_index = keras.ops.cumsum(ones_mask, axis=-2)
col_index = keras.ops.cumsum(ones_mask, axis=-1)
mask = ~ keras.ops.greater_equal(row_index, col_index)
mask

tensor([[[False,  True,  True],
         [False, False,  True],
         [False, False, False]]], device='cuda:0')

In [39]:
# в слое с вниманием эта маска будет использоваться чтобы занулить близости с токенами из будущего
(mask * -100)

tensor([[[   0, -100, -100],
         [   0,    0, -100],
         [   0,    0,    0]]], device='cuda:0')

In [40]:
# эта функция также проверяет и паддинг и если он есть то и его тоже замаскирует
# 1 - это паддинг айди
create_look_ahead_mask(torch.LongTensor([[1,3,2]]))

tensor([[[[1., 1., 1.],
          [1., 0., 1.],
          [1., 0., 0.]]]], device='cuda:0')

In [41]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

In [42]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

### Позиционное кодирование

In [43]:
# в трансформер передается информация о порядке слов
# Класс PositionalEncoding генерирует вектор для каждой позиции заданной размерности.
# Размерность задается также через d_model, чтобы можно было просто прибавить к вектору слова

class PositionalEncoding(keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / keras.ops.power(10000, (2 * (i // 2)) / d_model)
        return keras.ops.multiply(position, angles)

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=torch.arange(0, position, dtype=torch.float32)[:, None],
            i=torch.arange(0, d_model, dtype=torch.float32)[None, :],
            d_model=d_model)
        sines = keras.ops.sin(angle_rads[:, 0::2])
        cosines = keras.ops.cos(angle_rads[:, 1::2])

        pos_encoding = keras.ops.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[None, ...]
        return keras.ops.cast(pos_encoding, torch.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :inputs.shape[1], :]

### Encoder

In [44]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = keras.layers.Dropout(rate=dropout)(attention)
    attention = keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [45]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = keras.Input(shape=(max_len,), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    #inputs (они тут называются outputs но это просто такой нейминг,
    # этот параметр передается в encoder_layer первым и encoder_layer будет считать его inputs)
    # outputs он тут называется для удобства, так как он будет перезаписываться
    # чтобы на вход следующему блоку подавать уже не эмбединги,
    # а то что получится как результат предыдущего блока
    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Decoder

In [46]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [47]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = keras.Input(shape=(max_len,), name='inputs')
    enc_outputs = keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### Модель целиком

Общая модель состоит из энкодера, декодера и еще 1 полносвязного слоя. Дополнительный последний слой нужен, чтобы из векторов получить вероятности классов (=слов)

In [48]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = keras.Input(shape=(max_len[0],), name="inputs")
    dec_inputs = keras.Input(shape=(max_len[1]-1,), name="dec_inputs")

    enc_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

    look_ahead_mask = keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

    dec_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                          vocab_size=vocab_size[0],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[0],
                        )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                          vocab_size=vocab_size[1],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[1]-1,
                        )(inputs=[dec_inputs, enc_outputs,
                                  look_ahead_mask, dec_padding_mask])

    outputs = keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [49]:
# на вход подаются не вероятности, а логиты, то есть просто результат полносвязного слоя без активации
L  = keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = keras.ops.cast(keras.ops.not_equal(y_true, PAD_IDX), torch.float32)
    loss = keras.ops.multiply(loss, mask)

    return keras.ops.mean(loss)

Определяем параметры модели. Есть стандартные наборы параметров для обучения маленькой, средней или большой модели. Их можно посмотреть в оригинальной статье про трансформер - https://arxiv.org/pdf/1706.03762.pdf

Мы возьмем маленький, т.к. он требует меньше ресурсов. НО для практической задачи есть смысл использовать модель побольше, есть статьи в которые показано, что большие трансформеры требуют даже меньше данных для обучения -  https://arxiv.org/pdf/2002.11794.pdf.

In [50]:
keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])


optimizer = keras.optimizers.Adam(
    0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    return keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = keras.callbacks.ModelCheckpoint('model_ruen.weights.h5',
                                            monitor='val_loss',
                                            verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

Размер батча здесь уже имеет большое значение: сликом маленький батч приведет к сильному увеличению времени на обучение, но сильно большим его поставить не получится, т.к. модель большая и быстро займет всю видеокарту.

In [51]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [52]:
# model.load_weights('model_ruen.weights.h5')

In [53]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train,
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=200,
             epochs=3,
             callbacks=[checkpoint]
             )

Epoch 1/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.0887 - loss: 1.6192
Epoch 1: val_loss improved from inf to 0.98794, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1704s 359ms/step - accuracy: 0.0887 - loss: 1.6191 - val_accuracy: 0.1461 - val_loss: 0.9879
Epoch 2/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - accuracy: 0.1472 - loss: 0.9732
Epoch 2: val_loss improved from 0.98794 to 0.88990, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1711s 360ms/step - accuracy: 0.1472 - loss: 0.9732 - val_accuracy: 0.1580 - val_loss: 0.8899
Epoch 3/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - accuracy: 0.1585 - loss: 0.8749
Epoch 3: val_loss improved from 0.88990 to 0.85133, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1711s 360ms/step - accuracy: 0.1585 - loss: 0.8749 - val_accuracy: 0.1630 - val_loss: 0.8513


###Оценка качества модели вручную

In [76]:
@torch.no_grad()
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32)



    output_ids = [tokenizer_ru.token_to_id('[START]') ]

    pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()


    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]')]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()

    return tokenizer_ru.decode(output_ids[1:], )


In [55]:
translate("Transformer")

'трансформатортортортертеререртеререртеререр'

In [56]:
translate("can you translate this sentence?")

'вы можете перевести это?'

In [57]:
translate("please translate this sentence into russian")

'пожалуйста перевод на,, пожалуйста,,,,,,, пожалуйста,,,,,,,,,,,,, пожалуйста,,,,,,,,, пожалуйста,,,,.'

### BLEU

Реализация bleu есть в nltk

In [58]:
# %pip install nltk

In [59]:
import nltk

In [60]:
text = open('opus.en-ru-test.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-test.ru', 'w')
f.write(text)
f.close()

In [61]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [62]:
en_sents_test[:10]

['if you only stay there.',
 "i don't know how you do it, pop, carrying these boxes around every day.",
 'we might have a slight edge in mediation.',
 'how long is it going to take you to get him what he needs?',
 "on 1 april president of the nagorno karabagh republic bako sahakyan met head of the general staff of the republic of armenia's armed forces colonel-general yuri khachaturov.",
 'mr priesner also noted that the e-justice management system has not only improved case management, but has also led to a significant streamlining in procedures.',
 "you don't like chicken noodle soup?",
 'posted: 14 may 2005, 20:31',
 'now, for a minute, i thought maybe he was being tailed.',
 '« : 26 октябрь 2017, 06:50:24 »']

In [63]:
ru_sents_test[:10]

['только бы не вылететь.',
 'и как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
 'возможно, у нас есть небольшое преимущество в переговорах.',
 'сколько времени вы будете делать то, что ему нужно?',
 '1 апреля президент нкр бако саакян принял начальника генштаба вооруженных сил республики армения генерал-полковника юрия хачатурова.',
 'г-н приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упорядочению процедур.',
 '- неплохо, да.',
 'posted: 15 dec 2006, 00:07',
 'и на минутку я подумал, что за ним могут следить.',
 '«: 11 октябрь 2011, 17:15:34»']

In [64]:
# предсказание переводов для всех предложений в тестовой выборке
translations = [translate(en_sent) for en_sent in en_sents_test]

# рассчитываем BLEU для каждого предложения
bleu_scores = []
for i, translation in enumerate(translations):
    reference = tokenizer_ru.encode(ru_sents_test[i]).tokens
    hypothesis = tokenizer_ru.encode(translation).tokens
    bleu_score = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
    bleu_scores.append((i, bleu_score))

# сортируем предложения по BLEU
sorted_bleu_scores = sorted(bleu_scores, key=lambda x: x[1], reverse=True)

# функция для вывода лучших переводов согласно BLEU
def print_top_translations(sorted_bleu_scores, translations, n=20):
    print("Топ", n, "переводов:")
    for i in range(n):
        index, bleu_score = sorted_bleu_scores[i]
        print("BLEU score:", bleu_score)
        print("Английское предложение:", en_sents_test[index])
        print("Правильный перевод:", ru_sents_test[index])
        print("Наш перевод:", translations[index])
        print()

print_top_translations(sorted_bleu_scores, translations)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Топ 20 переводов:
BLEU score: 1.0
Английское предложение: 12844
Правильный перевод: 12844
Наш перевод: 12844

BLEU score: 1.0
Английское предложение: s5000
Правильный перевод: s5000
Наш перевод: s5000

BLEU score: 1.0
Английское предложение: published: 24 october 2014
Правильный перевод: опубликовано: 24 октябрь 2014
Наш перевод: опубликовано : 24 октябрь 2014

BLEU score: 1.0
Английское предложение: spain ( 15 apartments) apartments
Правильный перевод: испания ( 15 апартаментов) апартаментов
Наш перевод: испания ( 15 апартаментов ) апартаментов

BLEU score: 1.0
Английское предложение: jon: what is this?
Правильный перевод: что это?
Наш перевод: что это?

BLEU score: 1.0
Английское предложение: 6624
Правильный перевод: 6624
Наш перевод: 6624

BLEU score: 0.668740304976422
Английское предложение: sound: [k]
Правильный перевод: звук: [k]
Наш перевод: звуко : [ k ]

BLEU score: 0.6115380576901023
Английское предложение: cincinnati (ohio) 513231 **** mobile
Правильный перевод: цинциннати (

In [69]:
bleu_scores_values = [score for _, score in bleu_scores]

average_bleu_score = (sum(bleu_scores_values) / len(bleu_scores_values)) * 100
print("Средний BLEU-скор:", average_bleu_score)


Средний BLEU-скор: 2.0443447847407654


BLEU обычно показывают не от 0 до 1, а от 0 до 100.

### Оптимизация функции translate

In [126]:
@torch.no_grad()
def translate_batch(texts, batch_size=3):
    translations = []
    nb_samples = len(texts)
    for idx in range(0, nb_samples, batch_size):
        batch_texts = texts[idx:idx+batch_size]
        batch_translations = translate_parallel(batch_texts)
        translations.extend(batch_translations)
    return translations

def translate_parallel(batch_texts):
    input_ids = [encode(text, tokenizer_en, target=False) for text in batch_texts]
    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
        input_ids, maxlen=max_len_en, padding='post',
        value=PAD_IDX), torch.int32)

    output_ids = [[tokenizer_ru.token_to_id('[START]')]] * len(batch_texts)
    pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
        output_ids, maxlen=max_len_ru-1, padding='post',
        value=PAD_IDX), torch.int32))).cpu().numpy()

    while not all(pred.argmax(2)[:, -1] == tokenizer_ru.token_to_id('[END]')):
        if len(output_ids[0]) >= max_len_ru:
            break
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100
        next_words = pred.argmax(2)[:, -1]
        output_ids = [output + [word] if word != tokenizer_ru.token_to_id('[END]') else output for output, word in zip(output_ids, next_words)]
        pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
            output_ids, maxlen=max_len_ru-1, padding='post',
            value=PAD_IDX), torch.int32))).cpu().numpy()

    return [tokenizer_ru.decode(output[1:]) for output in output_ids]


In [132]:
texts = [
    "This is the first sentence.",
    "If you only stay there.",
    "We might have a slight edge in mediation.",
    "I don't know how you do it, pop, carrying these boxes around every day.",
    "please translate this sentence into russian",
    "can you translate this sentence?",
    "And finally the last sentence"
]

# размер батча
batch_size = 7

translations = translate_batch(texts, batch_size)

for text, translation in zip(texts, translations):
    print("Оригинал:", text)
    print("Перевод:", translation)
    print()


Оригинал: This is the first sentence.
Перевод: это первое предложение.

Оригинал: If you only stay there.
Перевод: если ты останешься там.. если ты там? не?.. если ты не?.

Оригинал: We might have a slight edge in mediation.
Перевод: у нас нас собеседование по делам посредни. может?. нам может. может........... нам может. может..........

Оригинал: I don't know how you do it, pop, carrying these boxes around every day.
Перевод: я не знаю,, ли ты ты,, попи,и,ррииииииииии.!!!!!!! я не знаю,,,.....

Оригинал: please translate this sentence into russian
Перевод: пожалуйста перевод на,, пожалуйста,,,,,,, пожалуйста,,,,,,,,,,,,, пожалуйста,,,,,,,,, пожалуйста,,,,.

Оригинал: can you translate this sentence?
Перевод: вы можете перевести это?

Оригинал: And finally the last sentence
Перевод: и последнее предложение



In [1]:

import torch
import torch.nn.parallel
from torch.nn.parallel import DataParallel

@torch.no_grad()
def translate_batch(texts, batch_size=3):
    translations = []
    nb_samples = len(texts)
    for idx in range(0, nb_samples, batch_size):
        batch_texts = texts[idx:idx+batch_size]
        batch_translations = translate_parallel(batch_texts)
        translations.extend(batch_translations)
    return translations

def translate_parallel(batch_texts):
    device_ids = [0, 1]

    # обертка модели в DataParallel
    model_parallel = DataParallel(model, device_ids=device_ids)

    input_ids = [encode(text, tokenizer_en, target=False) for text in batch_texts]
    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
        input_ids, maxlen=max_len_en, padding='post',
        value=PAD_IDX), torch.int32)

    output_ids = [[tokenizer_ru.token_to_id('[START]')]] * len(batch_texts)
    pred = model_parallel((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
        output_ids, maxlen=max_len_ru-1, padding='post',
        value=PAD_IDX), torch.int32))).cpu().numpy()

    while not all(pred.argmax(2)[:, -1] == tokenizer_ru.token_to_id('[END]')):
        if len(output_ids[0]) >= max_len_ru:
            break
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100
        next_words = pred.argmax(2)[:, -1]
        output_ids = [output + [word] if word != tokenizer_ru.token_to_id('[END]') else output for output, word in zip(output_ids, next_words)]
        pred = model_parallel((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
            output_ids, maxlen=max_len_ru-1, padding='post',
            value=PAD_IDX), torch.int32))).cpu().numpy()

    return [tokenizer_ru.decode(output[1:]) for output in output_ids]



## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.



Техника Backtranslation применяется для улучшения качества машинного перевода, особенно в ситуациях, когда у нас недостаточно обучающих данных. Это разновидность аугментации данных, при которой берутся тексты того языка, на который мы хотим переводить, и переводятся на тот язык, с которого мы переводим. Полученные синтетические параллельные корпуса добавляются к исходным обучающим данным. Модель после этого генерирует переводы качественнее, потому что в аугментированных данных показано, как должны выглядеть правильные переводы на целевой язык.

Как применить к задаче en->ru перевода:
1. Обучить модель переводить с русского на английский на исходном параллельном корпусе.
2. Найти большой корпус с русскими текстами, используя нашу модель перевести их на английский. Так мы получим новый синтетический параллельный корпус, где английские тексты будут не очень хорошими, зато русские, целевые, очень хорошими.
3. Объединить синтетический корпус с исходным.
4. Обучить модель переводить с английского на русский на новом увеличенном корпусе.  

Итого, понадобится две модели, каждую надо будет обучить по одному разу.